# Logistic Regression with a Neural Network implemented as a neural network without hidden layers, where the output layer consists of a single unit followed by a sigmoid activation function


In [1]:
import numpy as np
import copy
import matplotlib.pyplot as plt

**Mathematical expression of the algorithm**:

For one example $x^{(i)}$

$z^{(i)} = w^T x^{(i)} + b $

$\hat{y}^{(i)} = a^{(i)} = sigmoid(z^{(i)})$

$\mathcal{L}(a^{(i)}, Y^{(i)})  = - y^{(i)} \log(a^{(i)} - (1-y^{(i)} )  \log(1-a^{(i)})$

The cost is then computed by summing over all training examples:

$J = \frac{1}{m} \sum_{i=1}^m \mathcal{L}(a^{(i)}, y^{(i)})$

**sigmoid function** 

 $sigmoid(z) = \frac{1}{1 + e^{-z}}$ for $z = w^T x + b$ to make predictions.

In [3]:
def sigmoid(z):
    s = 1 / (1 + np.exp(-z))
    return s

### Forward and Backward propagation

**Hints**:

Forward Propagation:
- We have X
- We compute $A = \sigma(w^T X + b) = (a^{(1)}, a^{(2)}, ..., a^{(m-1)}, a^{(m)})$
- We calculate the cost function: $J = -\frac{1}{m}\sum_{i=1}^{m}(y^{(i)}\log(a^{(i)})+(1-y^{(i)})\log(1-a^{(i)}))$

Here are the two formulas we will be using: 

$$ \frac{\partial J}{\partial w} = \frac{1}{m}X(A-Y)^T$$
$$ \frac{\partial J}{\partial b} = \frac{1}{m} \sum_{i=1}^m (a^{(i)}-y^{(i)})$$

In [4]:

def propagate(w, b, X, Y):
    """
    Implement the cost function and its gradient for the propagation explained above
    
    """
    
    m = X.shape[1]
    
    Z = np.dot(w.T, X) + b
    A = sigmoid(Z)
    L = Y * np.log(A) + (1 - Y) * np.log(1 - A)
    cost = (-1 / m) * (np.sum(L, axis = 1, keepdims=True))
    
    dw = (1/m) * (np.dot(X, (A - Y).T))
    db = (1 / m) * np.sum(A - Y)
    
    cost = np.squeeze(np.array(cost))
    
    grads = {"dw": dw,
             "db": db}
    
    return grads, cost

**Optimization**

We want to update the parameters using gradient descent.The goal is to learn $w$ and $b$ by minimizing the cost function $J$. For a parameter $\theta$, the update rule is $ \theta = \theta - \alpha \text{ } d\theta$, where $\alpha$ is the learning rate.ate.
ate.
ng rate.

In [5]:
def optimize(w, b, X, Y, num_iterations=100, learning_rate=0.009, print_cost=False):
    """
    This function optimizes w and b by running a gradient descent algorithm
    
    """
    
    w = copy.deepcopy(w)
    b = copy.deepcopy(b)
    
    costs = []
    
    for i in range(num_iterations):
        
        grads, cost = propagate(w, b, X, Y)
    
        dw = grads["dw"]
        db = grads["db"]
    
        w = w - learning_rate * dw
        b = b - learning_rate * db
        
        # Record the costs
        if i % 100 == 0:
            costs.append(cost)
        
            # Print the cost every 100 training iterations
            if print_cost:
                print ("Cost after iteration %i: %f" %(i, cost))
    
    params = {"w": w,
              "b": b}
    
    grads = {"dw": dw,
             "db": db}
    
    return params, grads, costs

**predict**

1 - Calculate $\hat{Y} = A = \sigma(w^T X + b)$

2 - Convert the entries of a into 0 (if activation <= 0.5) or 1 (if activation > 0.5), stores the predictions in a vector `Y_prediction`.

In [6]:
def predict(w, b, X):
    '''
    Predict whether the label is 0 or 1 using learned logistic regression parameters (w, b)

    '''
    
    m = X.shape[1]
    Y_prediction = np.zeros((1, m))
    w = w.reshape(X.shape[0], 1)
    
    Z = np.dot(w.T, X) + b
    A = sigmoid(Z)
    
    Y_prediction = np.where(A > 0.5, 1, 0)
    
    return Y_prediction

### Merge all functions into a model

In [7]:
def model(X_train, Y_train, X_test, Y_test, num_iterations=2000, learning_rate=0.5, print_cost=False):
    """
    Builds the logistic regression model by calling the function you've implemented previously
    
    Returns:
    d -- dictionary containing information about the model.
    """
    w, b = np.zeros((X_train.shape[0], 1)), 0.0
    
    params, grads, costs = optimize(w, b, X_train, Y_train, num_iterations, learning_rate, print_cost=False)
    
    w = params["w"]
    b = params["b"]
    Y_prediction_test = predict(w, b, X_test)
    Y_prediction_train = predict(w, b, X_train)

    # Print train/test Errors
    if print_cost:
        print("train accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_train - Y_train)) * 100))
        print("test accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_test - Y_test)) * 100))

    
    d = {"costs": costs,
         "Y_prediction_test": Y_prediction_test, 
         "Y_prediction_train" : Y_prediction_train, 
         "w" : w, 
         "b" : b,
         "learning_rate" : learning_rate,
         "num_iterations": num_iterations}
    
    return d